# Tag validation

In [ ]:
import pandas as pd
import pandas as pd
import json
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import itertools

datapath = "D:/Jihye/data/"


def data_load(datapath): #data 불러와서 dataframe으로 저장하는 함수

    #json파일 불러와서 dataframe화
    #with open(datapath+"genre_gn_all.json", encoding = 'utf-8') as json_file: genre = json.load(json_file)
    #with open(datapath+"song_meta.json", encoding = 'utf-8') as json_file: song_meta = json.load(json_file)
    with open(datapath+"train.json", encoding = 'utf-8') as json_file: train = json.load(json_file)
    with open(datapath+"val.json", encoding = 'utf-8') as json_file: valid = json.load(json_file)
    
    # json to dataframe
    #df_genre = pd.Series(genre).to_frame('genre').reset_index().rename({'index' : 'gen_id'}, axis = 1)
    #df_song_meta = pd.DataFrame(song_meta)
    df_train = pd.DataFrame(train)
    df_valid = pd.DataFrame(valid)
    tag_list=df_train['tags'].tolist()
    
    return tag_list, df_train, df_valid

def one_hot_encoding(tag_list):
    te = TransactionEncoder()
    te_ary = te.fit(tag_list).transform(tag_list)
    return pd.DataFrame(te_ary, columns=te.columns_)

def analysis_tags(result, min_support) :
    frequent_itemsets = apriori(result, min_support=min_support, use_colnames=True) #Apriori 적용, 전체에서 등장확률이 min_support이상인 itemset만 출력
    rules=association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    return frequent_itemsets, rules

def search_tags(tag_list,rules,i): #생성된 association_rules에서 원하는 tag를 검색, yield로 구현해보기
    playlist=tag_list[i]
    #print ("기존 태그")
    #print (playlist)
    A=rules[(rules.antecedents=={playlist[0]})] #rules에서 해당하는 태그만 데이터프레임으로 만들기
    for p in range(len(playlist)-1):
        try:
            B=rules[(rules.antecedents=={playlist[p+1]})]
            C=pd.concat([A,B],axis=0).reset_index(drop=True)
            A=C
        except: break 
    
    selected=A[['antecedents','consequents','support','confidence','lift']] #향상도에 내림차순 정렬
    #향상도 1이상, 신뢰도 0.1이상만 보여줌
    S=selected[(selected['lift']>1.0)& (selected['confidence']>0.2) ].sort_values(by=['lift'],axis=0,ascending=False).reset_index(drop=True)
    
    #print (S)
    taglist=[]
    
    for c in range(len(S.consequents)):
        T=list(S.consequents[c])
        taglist.append(T)
        
    taglist=list(itertools.chain.from_iterable(taglist))   

    Tag=[]
    for v in taglist:
        if ((v not in Tag) and (v not in playlist)): #중복을 제거하고, 기존태그와 겹치는 태그도 제거
            Tag.append(v)
            if len(Tag)+len(playlist)==10 : break #총합 10개의 태그만 Tag리스트에 넣음
                
    #print ("관련 태그")
    #print (Tag)
     
    return Tag

def main():

    tag_list, df_train, df_valid = data_load(datapath)
    result=one_hot_encoding(tag_list)
    frequent_itemsets, rules = analysis_tags(result, 0.001)
    frequent_itemsets.to_csv('frequent_itemsets')
    rules.to_csv('rules.csv')

    V=df_valid[['tags','id','songs']]

    for i in range(len(tag_list)):
        Tag=search_tags(tag_list,rules,i)
        if (V.tags[i]==[]):
            print ('{} : playlist id {} 태그 없음'.format(i,V.id[i]))
        else:
            print ('{} : playlist id {} 기존 태그 {} 추가 태그 {}'.format(i,V.id[i],V.tags[i],Tag))
            V.tags[i].extend(Tag)
    print (V)

if __name__== "__main__":
    main()

In [ ]:
main()

최소 support 0.001 (115개), 최소 confidence 0.2, 향상도 순으로 추가